# Financial screener Data - ICA P2

It's a demo project to show the possibilities of [Plotly Dash](https://dash.plotly.com/)

## Introduction

For the ICA part 2, I decided to make an application for screening and analyzing financial data of the currency exchange market. From my perspective one of the most demanded areas of data science - is Financial. The finance industry generates lots of data. Big data in finance refers to the petabytes of structured and unstructured data that can be used for the creation of strategies for business institutions.

In this notebook I want to describe the design, cleaning data approach, and code implementation steps of the application for the screen market prices application.

Here are some examples of real screening services:
 - [finviz](https://finviz.com/)
 - [finance.yahoo](https://finance.yahoo.com/quote/%5EGSPC?p=%5EGSPC)
 - [investing](https://www.investing.com/equities/apple-computer-inc)

Also, in this project, I'll use some principles of technical analysis of the price charts and some special terminology. Anyway, I'll try to describe everything in simple language for a non-technical person. But, if you don't have any understanding of what price charts in the market are, you would find all the basic information in this article: [Understanding Basic Candlestick Charts](https://www.investopedia.com/trading/candlestick-charting-what-is-it/)

Since it's only a demo project, we will create something not complicated but with all the main features needed for a real price screening application:

- Charts with different display options
- Slider with ability to change date range
- Indicator - another chart with some displayed price analyse
- Cleaned and transformed data frames with different currency pairs
- Cleaned and transformed data frames with different timeframes
- Possobility to retrieve data by year from data frame
- \[Optional\] Dark and light theme for the chart.

According to the requirements our design is:
![screener-designe](img/screener-designe.png)


## Data View

The most important thing is to find proper data. I decided to take real financial data from [BITSTAMP EXCHANGE DATA](https://www.cryptodatadownload.com/data/bitstamp). From a demo perspective, I took only a few currency pairs: <span style="color:orange">"BTCEUR"</span>, <span style="color:orange">"BTCUSD"</span>, <span style="color:orange">"ETHBTC"</span>, and <span style="color:orange">"ETHEUR"</span>. Each coin pair is represented by two data frames: a day timeframe, and an hour timeframe. In total, we have 8 data frames.

Let's import <span style="color:yellow">pandas</span> and take a look at one of the data frames:

In [1]:
import pandas as pd

df = pd.read_csv('data/BTCEUR_1h.csv')

df.head()

,,,,,,,,https://www.CryptoDataDownload.com
unix,date,symbol,open,high,low,close,Volume BTC,Volume EUR
1672790400,2023-01-04 00:00:00,BTC/EUR,15798,15802,15787,15788,1.65765029,26170.98277852
1672786800,2023-01-03 23:00:00,BTC/EUR,15783,15806,15783,15801,4.79777073,75809.57530473
1672783200,2023-01-03 22:00:00,BTC/EUR,15786,15789,15772,15783,3.24313136,51186.34225488
1672779600,2023-01-03 21:00:00,BTC/EUR,15775,15791,15770,15786,9.63655850,152122.712481


Looks like, we have a set of different prices and dates. We need to understand that every second and millisecond there is appeared a hundred transactions in the real market, with every currency pair. It's impossible to show all of the price movement in the chart, but we don't need it, it's quite enough to show the result of these transactions in the selected time frame. For example, let's get the first row in our data frame. We are interested in the <span style="color:yellow">open</span> and <span style="color:yellow">close</span> values. Here we can see that the open is 15798 and the close - is 15788, the close is lower than the open which means that we have a downtrend and the candle is red. The high of the candle is the difference between opening and closing. In other words, we can say if the difference is positive - it's a green candle, up trend, if negative - a red candle, downtrend.

Example of prices os represented by candles:
![candle-example](img/candles-example.png)

The thin lines above and under the candle body it's a range of prices in the different timeframes, in other words, the lowest and highest prices in the current timeframe. It's represented by <span style="color:yellow">high</span> and <span style="color:yellow">low</span> values in the data frame.

Another important value is <span style="color:yellow">date</span>, we'll use it to manage the order of candles in the chart

Other columns like <span style="color:yellow">unix</span>, <span style="color:yellow">Volume BTC</span>, <span style="color:yellow">Volume EUR</span> are not necessary for the chart view and we'll delete them during data cleaning.

## Data cleaning

Before we jump into the cleaning process, let’s take a brief look at the data.

In [2]:
# Find all null values by columns
print(df.isnull().mean())

https://www.CryptoDataDownload.com    0.0
dtype: float64


In [3]:
# Total rows and columns
print(df.shape)

(40676, 1)


In [4]:
# Take a look on first rows
df.head()

,,,,,,,,https://www.CryptoDataDownload.com
unix,date,symbol,open,high,low,close,Volume BTC,Volume EUR
1672790400,2023-01-04 00:00:00,BTC/EUR,15798,15802,15787,15788,1.65765029,26170.98277852
1672786800,2023-01-03 23:00:00,BTC/EUR,15783,15806,15783,15801,4.79777073,75809.57530473
1672783200,2023-01-03 22:00:00,BTC/EUR,15786,15789,15772,15783,3.24313136,51186.34225488
1672779600,2023-01-03 21:00:00,BTC/EUR,15775,15791,15770,15786,9.63655850,152122.712481


In [5]:
# Take a look on last rows
df.tail()

,,,,,,,,https://www.CryptoDataDownload.com
1526378400,2018-05-15 10:00:00,BTC/EUR,7316.1,7427.72,7296.07,7381.92,707318.62,96.02
1526374800,2018-05-15 09:00:00,BTC/EUR,7313.2,7347.22,7300.0,7316.1,685687.14,93.52
1526371200,2018-05-15 08:00:00,BTC/EUR,7339.67,7339.67,7265.4,7313.2,343404.76,47.06
1526367600,2018-05-15 07:00:00,BTC/EUR,7335.0,7354.32,7306.34,7339.67,181611.86,24.79
1526364000,2018-05-15 06:00:00,BTC/EUR,7311.2,7390.7,7307.57,7335.0,480389.46,65.38


From these results, we learn that the dataset has 40675 rows and 9 columns. Of course, it's only a <span style="color:yellow"/>1-hour</span> timeframe dataset, for the <span style="color:yellow"/>1-day</span> it would be fewer rows. We also identify that rows are sorted by date in descending order. And we double-checked that our data frame is already in consist state without null values. These are all useful information.

Now we can clean and transform data.

#### Skip ad link rows

The first row in all dataframes is the ad link. Let's drop it in the first place:

In [6]:
df = pd.read_csv('data/BTCEUR_1h.csv', skiprows=1)

#### Cut unnecessary date

In total, we have 40675 rows, and the range of all date is from 2023 to 2018 year. I want to use a shorter range for the demo project. I want to take only the 2019 - 2023 range:

In [7]:
df = df[df['date'] > '2018-31-12']

#### Reverse data frame

Values of date column is ordered in the descent, which is not we need. We want to show price history in the chart chart from the oldest in the left corner to newest in the right conner of chart:

In [8]:
df = df.iloc[::-1]

#### Drop unnecessary columns

Also, as we mentioned before we don't need all columns in data frame to build chart. Let's leave only necessary: <span style="color:yellow"/>data</span>, <span style="color:yellow"/>open</span>, <span style="color:yellow"/>close</span>, <span style="color:yellow"/>high</span>, <span style="color:yellow"/>low</span>:

In [9]:
df.drop(['unix', 'Volume {}'.format('BTCEUR'[:3]),
            'Volume {}'.format('BTCEUR'[3:6])], axis=1, inplace=True)

Now we have cleaned and transformed data.

## Code implementation

Already we have the design and we know how data should be cleaned. In this part, we're going to collect all together and implement the design and cleaned data methods into a functional application. For this prospect we need to split the implementation process into logical parts, which are:

 - Create Data cleaning methods.
 - Create methods for HTML tag creation.
 - Create necessary HTML elements.
 - Add logic methods for chart and index updates.

#### Data cleaning and transformation methods

We have 8 different data frames that needed to be cleaned. Also, we need to keep in mind that in real life it could be a hundred data frames. So, our data cleaning methods should be agile as possible.

First let's create a method that takes the name of the data frame, then retrieve it and cleans it.

In [10]:
def clear_table(table_name):
    
    # skip first row with ad link
    data = pd.read_csv('data/{}.csv'.format(table_name), skiprows=1)

    # delete unnecassery columns
    data.drop(['unix', 'Volume {}'.format(table_name[:3]),
              'Volume {}'.format(table_name[3:6])], axis=1, inplace=True)

    # reverse data frames
    data = data.iloc[::-1]

    # remove unnecessary data. we need only date starts on 2019
    data = data[data['date'] > '2018-31-12']

    return data

The better way to achieve it is to create a list of data frames and clean it in the loop:

In [11]:
def clear_data(array):
    data = {}

    for elem in array:
        data[elem] = clear_table(elem)

    return data

In [12]:
data = clear_data([
    "BTCEUR_1h",
    "BTCEUR_d",
    "BTCUSD_1h",
    "BTCUSD_d",
    "ETHBTC_1h",
    "ETHBTC_d",
    "ETHEUR_1h",
    "ETHEUR_d"
])

Now we have a dictionary with cleaned data. We'll use it in the next parts.

#### Methods for HTML creation

In this part, we're going to start the creation of basic HTML elements for our application. I don't want to implement CSS styles as well, for running applications, it's not necessary. But in the python file attached to this notebook, you'll find the same methods and HTML tags, but with CSS style implemented in an inline way.

According to the design, we have dropdowns, radio buttons, and sliders. For these, we need to import a new library - dash. Let's implement it step by step.

In [13]:
from dash import dcc, html

In [14]:
def create_dropdown(title, options, id, value):
    
    return html.Div([
        html.P(title),
        dcc.Dropdown(options, id=id, value=value)
    ])

In [15]:
def create_radiobutton(options, id, value):
    return html.Div([
        dcc.RadioItems(options, id=id, value=value)
    ])

In [16]:
def create_slider():
    return html.Div([
        html.P(id="current_range", children='You current range is:'),

        # Slider with initaila values
        html.Div([
            dcc.RangeSlider(0, 30, 1, value=[
                0, 1000], marks=None, id="range_slider")
        ], id="range_slider_container"),
    ])

    Pay attention, in the case of dropdown and radio buttons we created a method for generating new elements. But in the case of the slider, it's only a block of code, that is invoked by the method.

#### Base HTML elements implementaion

Now we are ready to collect all created methods together and start to build our application frame.

In [17]:
from dash import Dash

app = Dash()
app.title = "PF-ICA2"

app.layout = html.Div([
    html.H1("Financial data screener"),

    html. H3(
        "It's a demo project of plotly dash possobilityes, only for study prospects."),

    html.Div([
        create_dropdown('Currency',
                        [
                            {'label': 'BTC/EUR', 'value': 'btceur'},
                            {'label': 'BTC/USD', 'value': 'btcusd'},
                            {'label': 'ETH/BTC', 'value': 'ethbtc'},
                            {'label': 'ETH/EUR', 'value': 'etheur'},
                        ],
                        'coin_pair',
                        'btceur'),

        create_dropdown('Timeframe', ['day', 'hour'], 'timeframe', 'day'),

        create_dropdown(
            'Select a year',
            [2022, 2021, 2020, 2019],
            'select_year',
            2022),

    ]),

    create_slider(),

    html.Div([
        create_radiobutton(
            [
                {'label': 'Candles', 'value': 'candles'},
                {'label': 'OHLC', 'value': 'ohlc'},
                {'label': 'Line', 'value': 'line'}
            ],
            'chart_type',
            'candles'
        ),

        # plotly_dark - template for dark theme
        # plotly - template for light theme
        create_radiobutton(
            [
                {'label': 'Dark', 'value': 'plotly_dark'},
                {'label': 'Light', 'value': 'plotly'}
            ],
            'template',
            'plotly'
        ),

    ]),

    dcc.Graph(id="candles"),
    dcc.Graph(id="indicator"),
])

Before adding some logic to control elements, we need to create a method to retrieve the necessary data frame from the library:

In [18]:
def get_filter_data(coin_pair, timeframe, select_year):
    
    # we need to choose which data frame we'll use
    if coin_pair == 'btceur':
        df = data["BTCEUR_1h"], data["BTCEUR_d"]
    elif coin_pair == 'btcusd':
        df = data["BTCUSD_1h"], data["BTCUSD_d"]
    elif coin_pair == 'ethbtc':
        df = data["ETHBTC_1h"], data["ETHBTC_d"]
    else:
        df = data["ETHEUR_1h"], data["ETHEUR_d"]

    # since we have 2 timeframes for each frame we need to chose which one we take
    if timeframe == 'hour':
        df = df[0]
    else:
        df = df[1]

    # chose the range of data
    filtered_df = df.loc[(df['date'] > '{}-12-31 00:00:00'.format(select_year-1))
                         & (df['date'] < '{}-01-01 00:00:00'.format(select_year+1))]

    return filtered_df

#### Logic implementation

And now we can add logic. According to our design control elements, like radio buttons and dropdowns used to manage data displayed in charts and sliders. And slider in turn used for managing chart data as well, that's why we can't place all logic in one method, but instead need to have a separate method for updating the slider and separate for updating the chart.

Let's implement <span style="color:yellow">update_slider</span> method first:

In [19]:
from dash import Input, Output

@app.callback(
    Output("range_slider_container", "children"),
    Input("coin_pair", "value"),
    Input("timeframe", "value"),
    Input("select_year", "value"),
)
def update_slider(coin_pair, timeframe, select_year):
    filtered_df = get_filter_data(coin_pair, timeframe, select_year)
    
    return dcc.RangeSlider(
        min=0,
        max=int(len(filtered_df)),
        step=1,
        marks={
            0: {'label': filtered_df.iloc[0].date.split(" ")[0]},
            int(len(filtered_df))*0.25: {'label': filtered_df.iloc[int((len(filtered_df) - 1)*0.25)].date.split(" ")[0]},
            int(len(filtered_df))*0.5: {'label': filtered_df.iloc[int((len(filtered_df) - 1)*0.5)].date.split(" ")[0]},
            int(len(filtered_df))*0.75: {'label': filtered_df.iloc[int((len(filtered_df) - 1)*0.75)].date.split(" ")[0]},
            int(len(filtered_df)): {'label': filtered_df.iloc[int(len(filtered_df))-1].date.split(" ")[0]}
        },
        value=[0, int(len(filtered_df))],
        id="range_slider")

Here we used two additional elements from the Dash library, like Input and Output. Also, I want to split the slider into quarters by using marks, that's why we have this part:
```python
marks = {
    0: {'label': filtered_df.iloc[0].date.spl(" ")[0]},
    int(len(filtered_df))*0.25: {'label'filtered_df.iloc[int((len(filtered_df) - *0.25)].date.split(" ")[0]},
    int(len(filtered_df))*0.5: {'label'filtered_df.iloc[int((len(filtered_df) - *0.5)].date.split(" ")[0]},
    int(len(filtered_df))*0.75: {'label'filtered_df.iloc[int((len(filtered_df) - *0.75)].date.split(" ")[0]},
    int(len(filtered_df)): {'label': filtered_diloc[int(len(filtered_df))-1].date.split(" [0]}
}
```

And <span style="color:yellow">update_chart</span> method:

In [20]:
import plotly.graph_objects as go
import pandas_ta as ta
import plotly.express as px

@app.callback(
    Output("candles", "figure"),
    Output("current_range", "children"),
    Output("indicator", "figure"),
    Input("coin_pair", "value"),
    Input("timeframe", "value"),
    Input("range_slider", "value"),
    Input("chart_type", "value"),
    Input("select_year", "value"),
    Input("template", "value")
)
def update_figure(coin_pair, timeframe, range_slider, chart_type, select_year, template):

    filtered_df = get_filter_data(coin_pair, timeframe, select_year)

    # Here we choose the length of data frame
    ranged_df = filtered_df.iloc[range_slider[0]:range_slider[1]]

    message = "You current range is: {} - {}".format(ranged_df.iloc[0]['date'].split(
        " ")[0], ranged_df.iloc[-1]['date'].split(" ")[0])

    if chart_type == 'candles':
        figure = go.Figure(
            data=[
                go.Candlestick(
                    x=ranged_df['date'],
                    open=ranged_df['open'],
                    high=ranged_df['high'],
                    low=ranged_df['low'],
                    close=ranged_df['close']
                )
            ]
        )
    elif chart_type == 'ohlc':
        figure = go.Figure(
            data=[
                go.Ohlc(
                    x=ranged_df.date,
                    open=ranged_df.open,
                    high=ranged_df.high,
                    low=ranged_df.low,
                    close=ranged_df.close
                )
            ]
        )
    else:
        figure = go.Figure(
            [go.Scatter(x=ranged_df.date, y=ranged_df.high)])

    figure.update_layout(xaxis_rangeslider_visible=False,
                         height=500, template=template)

    # Indicator starts here
    ranged_df['rsi'] = ta.rsi(ranged_df.close.astype('float'))

    # Indicator starts after 14th row
    ranged_df = ranged_df.iloc[14:]

    indicator = px.line(x=ranged_df.date, y=ranged_df['rsi'])

    indicator.update_layout(height=250, template=template, yaxis={
                            'title': ""}, xaxis={'title': ""})

    return figure, message, indicator

Probably the most complicated part of this application. Here we use new libraries, such as:
 - pandas_ta - library for indicator displaying
 - plotly.graph_objects - to build candlesticks and bars chart
 - plotly.express - to build simple line chart

The logic is simple, according to dropdowns we chose which data frame to take and how to filter it. According to radio buttons we chose which chart and theme to use. There are three different charts available: "Candlesticks", "Bars", and "Line" and two themes: Dark and Light.

Also, I decided to keep all indicator logic here, only because indicators use the same filtered and ranged data as a chart, and I didn't want to duplicate code with heavy calculation processes in another method when we can do it in one place.

Another interesting point about indicators in the technical analysis methods is, the indicator by default takes the first 14 rows of data, and makes a prediction according to it, in another word indicator starts only from the 14th row of the data frame. that's why we have this part of the code:

```python
    ranged_df = ranged_df.iloc[14:]

    indicator = px.line(x=ranged_df.date, y=ranged_df['rsi'])
```

The last part, code to launch application:

In [ ]:
if __name__ == '__main__':
    app.run_server()

We've already described the design, data cleaning, and implementation of applications. The working python file with all described methods I'll attach to this report. Also, the deployment process, architecture of the application package, and also choice of tools I'll describe in detail in the technical report, also attached to this notebook.